# Branch and Bound Algorithm for Solving Mixed Integer Programs (MIPs) or Binary Integer Programs (BIPs)

- The idea of the Branch & Bound algorithm is to indirectly arrive at an integer solution by running multiple linear programs that do not have constraints on the integer variables.  Multiple constraints are successively added that in some cases force solutions to have integer decision variables.  
- Integer solutions thus obtained may or may not be an optimal solution because the constraints break the preceding model into two parts: the other subpart might have a better solution.  These integer solutions are, therefore, a lower bound (__bound__) on the best possible solution.
- Non-integer solutions for a subproblem are possible upper bounds on the best possible objective function value if the solution is less than previous upper bounds (__bound__).
- Breaking an existing problem into two with two mutually exclusive but exhaustive constraints is called __branching__.
- Termination of a branch: __pruning__
  - A branch does not need to investigated further if its solution, either integer or not, when its solution is less than the current lower bound
  - A branch does not need to be investigated further if the problem is infeasible: no other subproblems on that branch will be feasible.
- Termination of the algorithm
  - The optimal solution has been found when the current integer solution has an objective function (lower bound) greater than the solutions on all other branches (which, of course, have been pruned).ted
ted


# Linear Relaxation

The B&B Algorithm starts with solving a linear program without any integrality constraints on the integer decision variables.

In [56]:
import gurobipy as grb
import numpy as np

A = np.genfromtxt('A.txt')
b = np.genfromtxt('b.txt').reshape(3,1)
c = np.genfromtxt('c.txt')

# Create Gurobi model
m = grb.Model('Sherwood')

# Create matrix of decision variables
x = m.addMVar(shape=(2,1), vtype=grb.GRB.CONTINUOUS, name='x')

# Update model to include variables
m.update()

# Create constraints
m.addConstrs((A @ x <= b for i in range(1)), name='cnstr') # generator iterator required despite not needing it
''' Alternative statement with m.addMConstr() 
      - Requires a 1D numpy array '''
#m.addMConstr(A,x.reshape((2,)), grb.GRB.LESS_EQUAL, b, name='cnstr')

# Create objective function
m.setObjective(c @ x, grb.GRB.MAXIMIZE)

# Update model to include constraints and objective function
m.update()

# Optimize the model
m.optimize()

''' Print decision variable values and other information '''
for var in m.getVars():
    print(f'Variable: {var.varName}, Optimal Value = {var.x}, (LB,UB) = ({var.lb}, {var.ub})')

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Xeon(R) CPU E5-1650 v3 @ 3.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3 rows, 2 columns and 5 nonzeros
Model fingerprint: 0x022e68d5
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 2e+02]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.0000000e+02   1.336900e+01   0.000000e+00      0s
       2    7.0769231e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  7.076923077e+02
Variable: x[0,0], Optimal Value = 18.46153846153846, (LB,UB) = (0.0, inf)
Variable: x[1,0], Optimal Value = 15.384615384615387, (LB,UB) = (0.0, i

# Build a Function to Create/Run the Base LP Relaxation

We will need to run many optimizations based on this basic LP relaxation and so it makes sense to write a function that can build and return this model.  The function returns the Gurobi model (<code>m</code>) and the decision variables (<code>x</code>) that are needed to add constraints in the branch and bound algorithm.

In [46]:
def run_lp_relaxation():
    import gurobipy as grb
    import numpy as np
    
    A = np.genfromtxt('A.txt')
    b = np.genfromtxt('b.txt').reshape(3,1)
    c = np.genfromtxt('c.txt')
    
    # Create Gurobi model
    m = grb.Model('Sherwood')
    
    # Specify how to optimize and time limit (seconds)
    m.setParam('TimeLimit',7200)
    
    # Create matrix of decision variables
    x = m.addMVar(shape=(2,1), vtype=grb.GRB.CONTINUOUS, name='x')
    
    # Update model to include variables
    m.update()
    
    # Create constraints
    m.addConstrs((A @ x <= b for i in range(1)), name='cnstr') # generator iterator required despite not needing it
    ''' Alternative statement with m.addMConstr() 
          - Requires a 1D numpy array '''
    #m.addMConstr(A,x.reshape((2,)), grb.GRB.LESS_EQUAL, b, name='cnstr')
    
    # Create objective function
    m.setObjective(c @ x, grb.GRB.MAXIMIZE)
    
    # Update model to include constraints and objective function
    m.update()
    
    # Optimize the model
    #m.optimize()

    return m,x

In [47]:
m,x = run_lp_relaxation()

Set parameter TimeLimit to value 7200


In [48]:
m,x

(<gurobi.Model Continuous instance Sherwood: 3 constrs, 2 vars, Parameter changes: TimeLimit=7200.0, Username=(user-defined)>,
 <MVar (2, 1)>
 array([[<gurobi.Var x[0,0]>],
        [<gurobi.Var x[1,0]>]]))

# Branch 1 on $x_s$

## Add Constraint: $x_s \leq 18$

In [50]:
m,x = run_lp_relaxation()

m.addConstr(np.array([1.0, 0.0]) @ x <= 18)
m.update()
m.optimize()

''' Print decision variable values and other information '''
for var in m.getVars():
    print(f'Variable: {var.varName}, Optimal Value = {var.x}, (LB,UB) = ({var.lb}, {var.ub})')

Set parameter TimeLimit to value 7200
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Xeon(R) CPU E5-1650 v3 @ 3.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 4 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xab8de7e7
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.0000000e+02   1.825000e+01   0.000000e+00      0s
       1    7.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  7.000000000e+02
Variable: x[0,0], Optimal Value = 18.0, (LB,UB) = (0.0, inf)
Variable: x[1,0], Optimal Value = 16.0, (LB,UB

## Add Constraint: $x_s \geq 19$

In [51]:
m,x = run_lp_relaxation()

m.addConstr(np.array([1.0, 0.0]) @ x >= 19)
m.update()
m.optimize()

''' Print decision variable values and other information '''
for var in m.getVars():
    print(f'Variable: {var.varName}, Optimal Value = {var.x}, (LB,UB) = ({var.lb}, {var.ub})')

Set parameter TimeLimit to value 7200
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Xeon(R) CPU E5-1650 v3 @ 3.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 4 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xef7ba7df
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.0000000e+02   1.336900e+01   0.000000e+00      0s
       2    7.0500000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  7.050000000e+02
Variable: x[0,0], Optimal Value = 19.0, (LB,UB) = (0.0, inf)
Variable: x[1,0], Optimal Value = 13.5, (LB,UB

# Branch 2 on $x_c$ with $x_s \geq 19$

## Add Constraint: $x_c \leq 13$

In [52]:
m,x = run_lp_relaxation()

m.addConstr(np.array([1.0, 0.0]) @ x >= 19)
m.addConstr(np.array([0.0, 1.0]) @ x <= 13)
m.update()
m.optimize()

''' Print decision variable values and other information '''
for var in m.getVars():
    print(f'Variable: {var.varName}, Optimal Value = {var.x}, (LB,UB) = ({var.lb}, {var.ub})')

Set parameter TimeLimit to value 7200
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Xeon(R) CPU E5-1650 v3 @ 3.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 5 rows, 2 columns and 7 nonzeros
Model fingerprint: 0xac8e8a38
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+02]
Presolve removed 3 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.0000000e+02   1.336900e+01   0.000000e+00      0s
       3    7.0428571e+02   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds (0.00 work units)
Optimal objective  7.042857143e+02
Variable: x[0,0], Optimal Value = 19.142857142857142, (LB,UB) = (0.0, inf)
Variable: x[1,0], Optimal Value 

## Add Constraint: $x_c \geq 14$

In [53]:
m,x = run_lp_relaxation()

m.addConstr(np.array([1.0, 0.0]) @ x >= 19)
m.addConstr(np.array([0.0, 1.0]) @ x >= 14)
m.update()
m.optimize()

''' Print decision variable values and other information '''
for var in m.getVars():
    print(f'Variable: {var.varName}, Optimal Value = {var.x}, (LB,UB) = ({var.lb}, {var.ub})')

Set parameter TimeLimit to value 7200
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Xeon(R) CPU E5-1650 v3 @ 3.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 5 rows, 2 columns and 7 nonzeros
Model fingerprint: 0x2f8568de
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+02]
Presolve removed 3 rows and 0 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Infeasible or unbounded model


AttributeError: Unable to retrieve attribute 'x'

In [37]:

''' Print sensitivity analysis information on constraints '''
for con in m.getConstrs():
    print(f'{con.ConstrName} Constraint: {m.getRow(con)}, {con.Sense}, RHS = {con.RHS}, slack = {con.slack}') #, (LB,UB) = ({con.SARHSLow}, {con.SARHSUp})')

cnstr[0][0,0] Constraint: 4.0 x[0,0] + 3.0 x[1,0], <, RHS = 120.0, slack = 3.5
cnstr[0][1,0] Constraint: 7.0 x[0,0] + 2.0 x[1,0], <, RHS = 160.0, slack = 0.0
cnstr[0][2,0] Constraint: x[1,0], <, RHS = 32.0, slack = 18.5
R3 Constraint: x[0,0], >, RHS = 19.0, slack = 0.0
R4 Constraint: x[1,0], >, RHS = 14.0, slack = 0.5


# Branch 3 with $x_c \leq 13$
## Add Constraint $ x_s = 19$

In [54]:
m,x = run_lp_relaxation()

m.addConstr(np.array([1.0, 0.0]) @ x == 19)
m.addConstr(np.array([0.0, 1.0]) @ x <= 13)
m.update()
m.optimize()

''' Print decision variable values and other information '''
for var in m.getVars():
    print(f'Variable: {var.varName}, Optimal Value = {var.x}, (LB,UB) = ({var.lb}, {var.ub})')

Set parameter TimeLimit to value 7200
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Xeon(R) CPU E5-1650 v3 @ 3.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 5 rows, 2 columns and 7 nonzeros
Model fingerprint: 0xb845010d
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+02]
Presolve removed 5 rows and 2 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  7.000000000e+02
Variable: x[0,0], Optimal Value = 19.0, (LB,UB) = (0.0, inf)
Variable: x[1,0], Optimal Value = 13.0, (LB,UB) = (0.0, inf)


## Add Constraint $ x_s \geq 20$

In [55]:
m,x = run_lp_relaxation()

m.addConstr(np.array([1.0, 0.0]) @ x >= 20)
m.addConstr(np.array([0.0, 1.0]) @ x <= 13)
m.update()
m.optimize()

''' Print decision variable values and other information '''
for var in m.getVars():
    print(f'Variable: {var.varName}, Optimal Value = {var.x}, (LB,UB) = ({var.lb}, {var.ub})')

Set parameter TimeLimit to value 7200
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Xeon(R) CPU E5-1650 v3 @ 3.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 5 rows, 2 columns and 7 nonzeros
Model fingerprint: 0x64cfcea0
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+02]
Presolve removed 3 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.0000000e+02   1.336900e+01   0.000000e+00      0s
       2    7.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  7.000000000e+02
Variable: x[0,0], Optimal Value = 20.0, (LB,UB) = (0.0, inf)
Variable: x[1,0], Optimal Value = 10.0, (LB,UB

# Running Large MIPs/BIPs

Running a larger MIP model likely involves one of two strategies for stopping the algorithm in a reasonable amount of time:

- Setting an explicit time limit
- Setting a percentage difference from the best possible objective function value
  - Gurobi will stop the optimization when this termination threshold has been met or exceeded (when % deviation less than the threshold)

## Traveling Salesperson Problem (TSP)

In [18]:
import gurobipy as gpy
import numpy as np
import time
import itertools

def gen(n):
    its = []
    #print('Check for subtours: ', end=' ')
    for num in range(1,n):
        its.append(itertools.combinations(range(n), num))
        #print(num, end=' ')
    #return ((i,j) for i in range(n) for j in range(i+1,n) )
    return itertools.chain(*its)

#rlen = lambda x:range(len(x))
n = 12 # number of locations
dist_lb = 95
dist_ub = 120
dist_lrg = 99999

# Create/read (symmetric) distance matrix
filename = f'dist_{n}.txt'
try:
    dist = np.genfromtxt(filename)
except:
    dist = np.random.randint(dist_lb,dist_ub,size=(n,n))
    np.fill_diagonal(dist,0)
    dist = (dist + dist.T)/2
    #dist = np.array([dist[i][j] for i in range(n) for j in range(i+1,n)])
    np.savetxt(filename, dist)

start = time.time()

''' Create model '''
m = gpy.Model('TSP')
m.ModelSense = gpy.GRB.MINIMIZE

''' Termination Criteria: either time limit or error tolerance '''
# Specify how to optimize and time limit (seconds)
#m.setParam('TimeLimit',120)
m.setParam('MIPGap',0.05)
m.setParam('PreSolve',0.05)

''' Create dv for connections '''
#dv_conn = [(i,j) for i in range(n) for j in range(i)]
x = m.addMVar((n,n), vtype=gpy.GRB.BINARY,name='x') 
m.update()
print(f'Variables created ({time.time() - start} seconds)')

''' Create objective function '''
m.setObjective((x * dist).sum(), gpy.GRB.MINIMIZE)  
print(f'Objective function created ({time.time() - start} seconds)')

''' Constraint  to ensure each location has 1 route segment in and one out '''
# Alternate constraints using numpy syntax
#m.addConstrs((x.sum(axis=0) == np.ones(n) for i in range(1)))
#m.addConstrs((x.sum(axis=1) == np.ones(n) for i in range(1)))
m.addConstrs((np.ones(n) @ x == np.ones(n) for i in range(1)))
m.addConstrs((np.ones(n) @ x.transpose() == np.ones(n) for i in range(1)))

''' Subtour constraints '''
for t in gen(n):
    w = np.zeros(n)
    w[list(t)] = 1
    z = w.reshape(w.shape[0],1)
    #m.addLConstr(gpy.quicksum(x[t,:][:,t]), gpy.GRB.LESS_EQUAL, rhs=len(t)-1)
    m.addConstrs((w@x@z <= len(t)-1 for i in range(1)))
print(f'Constraints created ({time.time() - start} seconds)')


m.update()

m.write(f'tsp{n}.mps')

m.optimize()

print(f'Execution time: {time.time() - start} seconds')
m.printStats()

for var in m.getVars():
    if var.x >0:
      print(f'Variable: {var.varName}, Optimal Value = {var.x}, (LB,UB) = ({var.lb}, {var.ub})')

Set parameter MIPGap to value 0.05
Set parameter Presolve to value 0
Variables created (0.012001752853393555 seconds)
Objective function created (0.013002395629882812 seconds)
Constraints created (1.9198026657104492 seconds)
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Xeon(R) CPU E5-1650 v3 @ 3.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 4118 rows, 144 columns and 159888 nonzeros
Model fingerprint: 0xa339102a
Variable types: 0 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 1285.5000000
Variable types: 0 continuous, 144 integer (144 binary)

Root relaxation: objective 1.227500e+03, 34 iterations, 0.02 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective